In [11]:
import datetime  # For datetime objects
import os.path  # To manage paths
import sys  # To find out the script name (in argv[0])

import backtrader as bt


src_path = '/Users/yifeining/Documents/GitHub/triplec-quant-trading/src'
if src_path not in sys.path:
    sys.path.insert(0, src_path)
    
from models import *
from strategy import *

In [6]:
from __future__ import (absolute_import, division, print_function,
                        unicode_literals)

import datetime  # For datetime objects
import os.path  # To manage paths
import sys  # To find out the script name (in argv[0])

import backtrader as bt
            
class TestStrategy(bt.Strategy):
    params = (
        ('exitbars', 5),
    )

    def log(self, txt, dt=None):
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
#         print(self.datas[0])
        self.dataclose = self.datas[0].close
        self.dataopen = self.datas[0].open
        self.datahigh = self.datas[0].high
        self.datalow = self.datas[0].low
        self.volume = self.datas[0].volume
        
#         print(self.dataclose[0], self.dataopen[0], self.datahigh[0], self.volume[0], '=====')
        
        self.order = None
        self.buyprice = None
        self.buycomm = None
        self.bar_executed = 0
#         self.sma = bt.indicators.SimpleMovingAverage(self.data)

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(
                    'BUY EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                    (order.executed.price,
                     order.executed.value,
                     order.executed.comm))

                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm
            else:  # Sell
                self.log('SELL EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                         (order.executed.price,
                          order.executed.value,
                          order.executed.comm))

            self.bar_executed = len(self)

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        self.order = None

    def notify_trade(self, trade):
        if not trade.isclosed:
            return

        self.log('OPERATION PROFIT, GROSS %.2f, NET %.2f' %
                 (trade.pnl, trade.pnlcomm))

    def next(self):
        # Simply log the closing price of the series from the reference
        self.log('Close, %.2f' % self.dataclose[0])

        # Check if an order is pending ... if yes, we cannot send a 2nd one
        if self.order:
            return

        # Check if we are in the market
        if not self.position:

            # Not yet ... we MIGHT BUY if ...
            if self.dataclose[0] < self.dataclose[-1]:
                    # current close less than previous close

                    if self.dataclose[-1] < self.dataclose[-2]:
                        # previous close less than the previous close

                        # BUY, BUY, BUY!!! (with default parameters)
                        self.log('BUY CREATE, %.2f' % self.dataclose[0])

                        # Keep track of the created order to avoid a 2nd order
                        self.order = self.buy()

        else:
            # Already in the market ... we might sell
            if len(self) >= (self.bar_executed + self.params.exitbars):
                # SELL, SELL, SELL!!! (with all possible default parameters)
                self.log('SELL CREATE, %.2f' % self.dataclose[0])

                # Keep track of the created order to avoid a 2nd order
                self.order = self.sell()


In [12]:
cerebro = bt.Cerebro(cheat_on_open=False)
# cerebro = bt.Cerebro(stdstats=False)
# cerebro.addobserver(bt.Observers.BuySell)

dir_path = '/Users/yifeining/Documents/GitHub/triplec-quant-trading'

datapath1 = os.path.join(dir_path, 'data', 'survivorship-free', 'AAPL.csv')
# datapath2 = os.path.join(dir_path, 'data', 'survivorship-free', 'GOOG.csv')

data1 = bt.feeds.YahooFinanceCSVData(
    dataname=datapath1,
    fromdate=datetime.datetime(2013, 1, 1),
    todate=datetime.datetime(2018, 12, 1),
    adjclose = False
    )
cerebro.adddata(data1)

# data2 = bt.feeds.YahooFinanceCSVData(
#     dataname=datapath2,
#     fromdate=datetime.datetime(2015, 1, 1),
#     todate=datetime.datetime(2015, 12, 1),
#     reverse=False,
#     adjclose = False)

# data2.compensate(data1)  # let the system know ops on data1 affect data0
# data2.plotinfo.plotmaster = data1
# data2.plotinfo.sameaxis = True
# cerebro.adddata(data2)

In [13]:
# Add a strategy
cerebro.addstrategy(TestStrategy)

# Set our desired cash start
cerebro.broker.setcash(2000.0)

# Add a FixedSize sizer according to the stake
cerebro.addsizer(bt.sizers.PercentSizer, percents = 100) ##### cheat-on-open

# Set the commission - 0.1% ... divide by 100 to remove the %
cerebro.broker.setcommission(commission=0.00)

print('Initial Portfolio Value: %.2f' % cerebro.broker.getvalue())
# Run over everything
cerebro.run()

# cerebro.run(runonce=False, stdstats=False, writer=True)


Initial Portfolio Value: 2000.00
2014-03-31, Close, 71.77
2014-03-31, BUY CREATE, 71.77
2014-04-01, Order Canceled/Margin/Rejected
2014-04-01, Close, 72.43
2014-04-02, Close, 72.55
2014-04-03, Close, 72.04
2014-04-04, Close, 71.11
2014-04-04, BUY CREATE, 71.11
2014-04-07, BUY EXECUTED, Price: 70.60, Cost: 1985.66, Comm 0.00
2014-04-07, Close, 70.00
2014-04-08, Close, 69.99
2014-04-09, Close, 70.91
2014-04-10, Close, 70.00
2014-04-11, Close, 69.48
2014-04-14, Close, 69.76
2014-04-14, SELL CREATE, 69.76
2014-04-15, SELL EXECUTED, Price: 69.57, Cost: 1985.66, Comm 0.00
2014-04-15, OPERATION PROFIT, GROSS -28.97, NET -28.97
2014-04-15, Close, 69.26
2014-04-16, Close, 69.40
2014-04-17, Close, 70.19
2014-04-21, Close, 71.02
2014-04-22, Close, 71.10
2014-04-23, Close, 70.17
2014-04-24, Close, 75.92
2014-04-25, Close, 76.48
2014-04-28, Close, 79.44
2014-04-29, Close, 79.20
2014-04-30, Close, 78.90
2014-04-30, BUY CREATE, 78.90
2014-05-01, Order Canceled/Margin/Rejected
2014-05-01, Close, 79.09

2017-03-08, Close, 137.87
2017-03-09, Close, 137.56
2017-03-09, BUY CREATE, 137.56
2017-03-10, Order Canceled/Margin/Rejected
2017-03-10, Close, 138.01
2017-03-13, Close, 138.07
2017-03-14, Close, 137.86
2017-03-15, Close, 139.32
2017-03-16, Close, 139.55
2017-03-17, Close, 138.86
2017-03-20, Close, 140.31
2017-03-21, Close, 138.71
2017-03-22, Close, 140.27
2017-03-23, Close, 139.78
2017-03-24, Close, 139.50
2017-03-24, BUY CREATE, 139.50
2017-03-27, BUY EXECUTED, Price: 138.26, Cost: 2682.99, Comm 0.00
2017-03-27, Close, 139.74
2017-03-28, Close, 142.64
2017-03-29, Close, 142.95
2017-03-30, Close, 142.76
2017-03-31, Close, 142.50
2017-04-03, Close, 142.54
2017-04-03, SELL CREATE, 142.54
2017-04-04, SELL EXECUTED, Price: 142.09, Cost: 2682.99, Comm 0.00
2017-04-04, OPERATION PROFIT, GROSS 74.32, NET 74.32
2017-04-04, Close, 143.60
2017-04-05, Close, 142.85
2017-04-06, Close, 142.50
2017-04-06, BUY CREATE, 142.50
2017-04-07, Order Canceled/Margin/Rejected
2017-04-07, Close, 142.18
2017-

In [14]:
cerebro.plot();


In [15]:
print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())

Final Portfolio Value: 2874.78
